# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    # print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [ ]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [ ]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [ ]:
# Create the table.
query = "DROP TABLE IF EXISTS events_by_session_id_and_items;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

query = """CREATE TABLE IF NOT EXISTS events_by_session_id_and_items
           (artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text, location text, sessionId int, song text, userId int,
            PRIMARY KEY (sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Populate the table.
def insertRecord(session, values):
    """
    Function to insert a record into the database. Expects a list of values.
    """
    query = """ 
    INSERT INTO events_by_session_id_and_items 
    (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    try:
        session.execute(query, values)
    except Exception as e:
        print(e)

def sanitizeValues(values):
    """
    This function sanitizes the inputs of the CSV files. 
    It converts all the numerical fields into the right type by parsing them using int and float.
    """
    #[artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId]
    values[3] = int(values[3])
    values[5] = float(values[5])
    values[8] = int(values[8])
    values[10] = int(values[10])
    return values 

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insertRecord(session, sanitizeValues(line))

In [ ]:
# Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
query = """
        select artist, song, length from events_by_session_id_and_items where sessionId=338 and itemInSession=4;
        """
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)
                    

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [ ]:
# Create the table.
query = "DROP TABLE IF EXISTS events_by_userid_and_sessionid;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

query = """CREATE TABLE IF NOT EXISTS events_by_userid_and_sessionid
           (artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text, location text, sessionId int, song text, userId int,
            PRIMARY KEY ((userId, sessionId), itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Populate the table.
def insertRecord(session, values):
    """
    Function to insert a record into the database. Expects a list of values.
    """
    query = """ 
    INSERT INTO events_by_userid_and_sessionid 
    (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    try:
        session.execute(query, values)
    except Exception as e:
        print(e)

def sanitizeValues(values):
    """
    This function sanitizes the inputs of the CSV files. 
    It converts all the numerical fields into the right type by parsing them using int and float.
    """
    #[artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId]
    values[3] = int(values[3])
    values[5] = float(values[5])
    values[8] = int(values[8])
    values[10] = int(values[10])
    return values 

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insertRecord(session, sanitizeValues(line))

In [ ]:
# Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query = "select artist, song, firstname, lastname from events_by_userid_and_sessionid where userId=10 and sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)
                    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [ ]:
# Create the table.
query = "DROP TABLE IF EXISTS events_by_song_and_sessionid;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

query = """CREATE TABLE IF NOT EXISTS events_by_song_and_sessionid
           (artist text, firstName text, gender text, itemInSession int, lastName text, length float, level text, location text, sessionId int, song text, userId int,
            PRIMARY KEY (song, sessionId, itemInSession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Populate the table.
def insertRecord(session, values):
    """
    Function to insert a record into the database. Expects a list of values.
    """
    query = """ 
    INSERT INTO events_by_song_and_sessionid 
    (artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    try:
        session.execute(query, values)
    except Exception as e:
        print(e)

def sanitizeValues(values):
    """
    This function sanitizes the inputs of the CSV files. 
    It converts all the numerical fields into the right type by parsing them using int and float.
    """
    #[artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId]
    values[3] = int(values[3])
    values[5] = float(values[5])
    values[8] = int(values[8])
    values[10] = int(values[10])
    return values 

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insertRecord(session, sanitizeValues(line))

In [ ]:
# Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "select artist, song, firstname, lastname from events_by_song_and_sessionid where song='All Hands Against His Own';"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

### Drop the tables before closing out the sessions

In [ ]:
tables = ["events_by_session_id_and_items", "events_by_userid_and_sessionid", "events_by_song_and_sessionid"]
for table in tables: 
    query = """
    DROP TABLE IF EXISTS {};
    """
    try:
        rows = session.execute(query.format(table))
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()